In [ ]:

# Plotly compatibility fix
def fix_plotly_data(data):
    """Convert range/arange objects to lists for Plotly compatibility"""
    if isinstance(data, range):
        return list(data)
    elif isinstance(data, np.ndarray):
        return data.tolist()
    elif hasattr(data, 'tolist'):
        return data.tolist()
    return data


In [ ]:
"""
Banking and Financial Markets Analysis - Kenya's Financial System Performance
============================================================================

Comprehensive analysis of Kenya's banking and financial markets including:
- Interest rate trends and analysis
- Money market operations
- Banking sector performance
- Financial inclusion metrics
- Monetary policy effectiveness
"""

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Statistical and modeling libraries
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

class BankingFinancialAnalyzer:
    """Advanced Banking and Financial Markets Analysis Engine"""
    
    def __init__(self):
        self.interest_rates = {}
        self.money_market_data = {}
        self.banking_data = {}
        self.payment_systems = {}
        
    def load_financial_data(self, data_path="data/raw/"):
        """Load all banking and financial datasets"""
        
        # Interest rates data
        try:
            self.cbr_rates = pd.read_csv(f"{data_path}Central Bank Rate (CBR).csv", skiprows=2)
            self.interbank_rates = pd.read_csv(f"{data_path}Interbank Rates  Volumes.csv", skiprows=2)
            self.commercial_rates = pd.read_csv(f"{data_path}Commercial Banks Weighted Average Rates ().csv", skiprows=2)
            print("✅ Loaded Interest Rates Data")
        except Exception as e:
            print(f"❌ Could not load Interest Rates Data: {e}")
            
        # Money market operations
        try:
            self.repo_operations = pd.read_csv(f"{data_path}Repo and Reverse Repo .csv", skiprows=2)
            self.horizontal_repo = pd.read_csv(f"{data_path}Horizontal Repo Market.csv", skiprows=2)
            self.treasury_bills = pd.read_csv(f"{data_path}Issues of Treasury Bills.csv", skiprows=2)
            self.treasury_bonds = pd.read_csv(f"{data_path}Issues of Treasury Bonds.csv", skiprows=2)
            print("✅ Loaded Money Market Data")
        except Exception as e:
            print(f"❌ Could not load Money Market Data: {e}")
            
        # Payment systems
        try:
            self.rtgs_data = pd.read_csv(f"{data_path}KEPSSRTGS.csv", skiprows=2)
            self.cheques_efts = pd.read_csv(f"{data_path}Cheques  EFTs.csv", skiprows=2)
            self.atm_pos_data = pd.read_csv(f"{data_path}Number of ATMs, ATM Cards,  POS Machines.csv", skiprows=2)
            self.transactions_data = pd.read_csv(f"{data_path}Number of Transactions.csv", skiprows=2)
            print("✅ Loaded Payment Systems Data")
        except Exception as e:
            print(f"❌ Could not load Payment Systems Data: {e}")
            
        # Government securities
        try:
            self.gov_securities = pd.read_csv(f"{data_path}Government Securities Auction and Maturities Schedule.csv", skiprows=2)
            self.discount_window = pd.read_csv(f"{data_path}Discount Window.csv", skiprows=2)
            print("✅ Loaded Government Securities Data")
        except Exception as e:
            print(f"❌ Could not load Government Securities Data: {e}")
    
    def clean_financial_data(self):
        """Clean and standardize financial data"""
        
        datasets = ['cbr_rates', 'interbank_rates', 'commercial_rates', 'repo_operations', 
                   'treasury_bills', 'treasury_bonds', 'rtgs_data', 'cheques_efts']
        
        for dataset_name in datasets:
            if hasattr(self, dataset_name):
                dataset = getattr(self, dataset_name)
                
                # Remove rows with all NaN values
                dataset = dataset.dropna(how='all')
                
                # Convert numeric columns
                numeric_cols = dataset.select_dtypes(include=[object]).columns
                for col in numeric_cols[1:]:  # Skip first column (usually time/period)
                    try:
                        dataset[col] = pd.to_numeric(dataset[col].astype(str).str.replace(',', ''), errors='coerce')
                    except:
                        pass
                
                setattr(self, dataset_name, dataset)
        
        return self
    
    def analyze_interest_rate_dynamics(self):
        """Analyze interest rate trends and relationships"""
        
        analysis = {}
        
        # CBR trend analysis
        if hasattr(self, 'cbr_rates'):
            cbr_df = self.cbr_rates.copy()
            numeric_cols = cbr_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                cbr_col = numeric_cols[0]  # Assume first numeric column is CBR
                
                analysis['cbr_stats'] = {
                    'current_rate': cbr_df[cbr_col].iloc[-1] if not cbr_df[cbr_col].isna().all() else None,
                    'average_rate': cbr_df[cbr_col].mean(),
                    'volatility': cbr_df[cbr_col].std(),
                    'min_rate': cbr_df[cbr_col].min(),
                    'max_rate': cbr_df[cbr_col].max(),
                    'trend': np.polyfit(range(len(cbr_df[cbr_col].dropna())), cbr_df[cbr_col].dropna(), 1)[0]
                }
        
        # Interbank rates analysis
        if hasattr(self, 'interbank_rates'):
            interbank_df = self.interbank_rates.copy()
            numeric_cols = interbank_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                # Calculate average interbank rate
                interbank_df['avg_rate'] = interbank_df[numeric_cols].mean(axis=1, skipna=True)
                
                analysis['interbank_stats'] = {
                    'current_rate': interbank_df['avg_rate'].iloc[-1],
                    'average_rate': interbank_df['avg_rate'].mean(),
                    'volatility': interbank_df['avg_rate'].std(),
                    'correlation_with_cbr': self._calculate_correlation('cbr_rates', interbank_df['avg_rate'])
                }
        
        # Commercial bank rates analysis
        if hasattr(self, 'commercial_rates'):
            commercial_df = self.commercial_rates.copy()
            numeric_cols = commercial_df.select_dtypes(include=[np.number]).columns
            
            # Separate lending and deposit rates
            lending_cols = [col for col in numeric_cols if 'lending' in col.lower() or 'loan' in col.lower()]
            deposit_cols = [col for col in numeric_cols if 'deposit' in col.lower() or 'saving' in col.lower()]
            
            if lending_cols and deposit_cols:
                commercial_df['avg_lending_rate'] = commercial_df[lending_cols].mean(axis=1, skipna=True)
                commercial_df['avg_deposit_rate'] = commercial_df[deposit_cols].mean(axis=1, skipna=True)
                commercial_df['interest_spread'] = commercial_df['avg_lending_rate'] - commercial_df['avg_deposit_rate']
                
                analysis['commercial_rates'] = {
                    'current_lending_rate': commercial_df['avg_lending_rate'].iloc[-1],
                    'current_deposit_rate': commercial_df['avg_deposit_rate'].iloc[-1],
                    'current_spread': commercial_df['interest_spread'].iloc[-1],
                    'average_spread': commercial_df['interest_spread'].mean(),
                    'spread_volatility': commercial_df['interest_spread'].std()
                }
        
        return analysis
    
    def analyze_money_market_operations(self):
        """Analyze money market operations and liquidity management"""
        
        operations_analysis = {}
        
        # Repo operations analysis
        if hasattr(self, 'repo_operations'):
            repo_df = self.repo_operations.copy()
            numeric_cols = repo_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                # Calculate total repo volumes
                repo_df['total_repo_volume'] = repo_df[numeric_cols].sum(axis=1, skipna=True)
                
                operations_analysis['repo_operations'] = {
                    'average_volume': repo_df['total_repo_volume'].mean(),
                    'volume_volatility': repo_df['total_repo_volume'].std(),
                    'trend': np.polyfit(range(len(repo_df['total_repo_volume'].dropna())), 
                                      repo_df['total_repo_volume'].dropna(), 1)[0],
                    'recent_volume': repo_df['total_repo_volume'].tail(5).mean()
                }
        
        # Treasury bills analysis
        if hasattr(self, 'treasury_bills'):
            tbills_df = self.treasury_bills.copy()
            numeric_cols = tbills_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                # Calculate total T-bills issuance
                tbills_df['total_issuance'] = tbills_df[numeric_cols].sum(axis=1, skipna=True)
                
                operations_analysis['treasury_bills'] = {
                    'average_issuance': tbills_df['total_issuance'].mean(),
                    'issuance_trend': np.polyfit(range(len(tbills_df['total_issuance'].dropna())), 
                                                tbills_df['total_issuance'].dropna(), 1)[0],
                    'recent_issuance': tbills_df['total_issuance'].tail(5).mean()
                }
        
        # Treasury bonds analysis
        if hasattr(self, 'treasury_bonds'):
            tbonds_df = self.treasury_bonds.copy()
            numeric_cols = tbonds_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                tbonds_df['total_bonds'] = tbonds_df[numeric_cols].sum(axis=1, skipna=True)
                
                operations_analysis['treasury_bonds'] = {
                    'average_issuance': tbonds_df['total_bonds'].mean(),
                    'issuance_trend': np.polyfit(range(len(tbonds_df['total_bonds'].dropna())), 
                                                tbonds_df['total_bonds'].dropna(), 1)[0]
                }
        
        return operations_analysis
    
    def analyze_payment_systems(self):
        """Analyze payment systems efficiency and growth"""
        
        payments_analysis = {}
        
        # RTGS analysis
        if hasattr(self, 'rtgs_data'):
            rtgs_df = self.rtgs_data.copy()
            numeric_cols = rtgs_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                rtgs_df['total_rtgs_value'] = rtgs_df[numeric_cols].sum(axis=1, skipna=True)
                
                payments_analysis['rtgs'] = {
                    'average_value': rtgs_df['total_rtgs_value'].mean(),
                    'growth_rate': np.polyfit(range(len(rtgs_df['total_rtgs_value'].dropna())), 
                                            rtgs_df['total_rtgs_value'].dropna(), 1)[0],
                    'volatility': rtgs_df['total_rtgs_value'].std()
                }
        
        # ATM and POS analysis
        if hasattr(self, 'atm_pos_data'):
            atm_df = self.atm_pos_data.copy()
            numeric_cols = atm_df.select_dtypes(include=[np.number]).columns
            
            # Separate ATM and POS data
            atm_cols = [col for col in numeric_cols if 'atm' in col.lower()]
            pos_cols = [col for col in numeric_cols if 'pos' in col.lower()]
            
            if atm_cols:
                atm_df['total_atms'] = atm_df[atm_cols].sum(axis=1, skipna=True)
                payments_analysis['atm_infrastructure'] = {
                    'current_atms': atm_df['total_atms'].iloc[-1],
                    'growth_rate': np.polyfit(range(len(atm_df['total_atms'].dropna())), 
                                            atm_df['total_atms'].dropna(), 1)[0]
                }
            
            if pos_cols:
                atm_df['total_pos'] = atm_df[pos_cols].sum(axis=1, skipna=True)
                payments_analysis['pos_infrastructure'] = {
                    'current_pos': atm_df['total_pos'].iloc[-1],
                    'growth_rate': np.polyfit(range(len(atm_df['total_pos'].dropna())), 
                                            atm_df['total_pos'].dropna(), 1)[0]
                }
        
        return payments_analysis
    
    def _calculate_correlation(self, dataset1_name, series2):
        """Helper function to calculate correlation between datasets"""
        if hasattr(self, dataset1_name):
            dataset1 = getattr(self, dataset1_name)
            numeric_cols = dataset1.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                series1 = dataset1[numeric_cols[0]]
                # Align series by length
                min_len = min(len(series1.dropna()), len(series2.dropna()))
                if min_len > 1:
                    return np.corrcoef(series1.dropna()[:min_len], series2.dropna()[:min_len])[0, 1]
        return None
    
    def create_financial_visualizations(self):
        """Create comprehensive financial market visualizations"""
        
        visualizations = {}
        
        # 1. Interest Rate Trends
        if hasattr(self, 'cbr_rates'):
            cbr_df = self.cbr_rates.copy()
            numeric_cols = cbr_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                fig_rates = go.Figure()
                
                # Add CBR trend
                fig_rates.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(cbr_df)),
                    y=cbr_df[numeric_cols[0]],
                    mode='lines+markers',
                    name='Central Bank Rate',
                    line=dict(color='red', width=3)
                ))
                
                # Add commercial rates if available
                if hasattr(self, 'commercial_rates'):
                    comm_df = self.commercial_rates.copy()
                    comm_numeric = comm_df.select_dtypes(include=[np.number]).columns
                    if len(comm_numeric) > 0:
                        comm_df['avg_rate'] = comm_df[comm_numeric].mean(axis=1, skipna=True)
                        fig_rates.add_trace(go.Scatter(
                            x=fix_plotly_data(range(len(comm_df)),
                            y=comm_df['avg_rate'],
                            mode='lines',
                            name='Commercial Bank Average Rate',
                            line=dict(color='blue', width=2)
                        ))
                
                fig_rates.update_layout(
                    title='Kenya Interest Rate Trends',
                    xaxis_title='Period',
                    yaxis_title='Interest Rate (%)',
                    template='plotly_white',
                    height=500
                )
                visualizations['interest_rates'] = fig_rates
        
        # 2. Money Market Operations
        if hasattr(self, 'treasury_bills') and hasattr(self, 'treasury_bonds'):
            tbills_df = self.treasury_bills.copy()
            tbonds_df = self.treasury_bonds.copy()
            
            tbills_numeric = tbills_df.select_dtypes(include=[np.number]).columns
            tbonds_numeric = tbonds_df.select_dtypes(include=[np.number]).columns
            
            if len(tbills_numeric) > 0 and len(tbonds_numeric) > 0:
                tbills_total = tbills_df[tbills_numeric].sum(axis=1, skipna=True)
                tbonds_total = tbonds_df[tbonds_numeric].sum(axis=1, skipna=True)
                
                fig_securities = go.Figure()
                fig_securities.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(tbills_total)),
                    y=tbills_total,
                    mode='lines',
                    name='Treasury Bills',
                    fill='tonexty',
                    line=dict(color='green')
                ))
                fig_securities.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(tbonds_total)),
                    y=tbonds_total,
                    mode='lines',
                    name='Treasury Bonds',
                    fill='tonexty',
                    line=dict(color='orange')
                ))
                
                fig_securities.update_layout(
                    title='Government Securities Issuance',
                    xaxis_title='Period',
                    yaxis_title='Value (Ksh Millions)',
                    template='plotly_white',
                    height=500
                )
                visualizations['government_securities'] = fig_securities
        
        # 3. Payment Systems Growth
        if hasattr(self, 'atm_pos_data'):
            atm_df = self.atm_pos_data.copy()
            numeric_cols = atm_df.select_dtypes(include=[np.number]).columns
            
            atm_cols = [col for col in numeric_cols if 'atm' in col.lower()]
            pos_cols = [col for col in numeric_cols if 'pos' in col.lower()]
            
            if atm_cols or pos_cols:
                fig_infrastructure = go.Figure()
                
                if atm_cols:
                    atm_total = atm_df[atm_cols].sum(axis=1, skipna=True)
                    fig_infrastructure.add_trace(go.Scatter(
                        x=fix_plotly_data(range(len(atm_total)),
                        y=atm_total,
                        mode='lines+markers',
                        name='ATMs',
                        line=dict(color='purple')
                    ))
                
                if pos_cols:
                    pos_total = atm_df[pos_cols].sum(axis=1, skipna=True)
                    fig_infrastructure.add_trace(go.Scatter(
                        x=fix_plotly_data(range(len(pos_total)),
                        y=pos_total,
                        mode='lines+markers',
                        name='POS Machines',
                        line=dict(color='cyan')
                    ))
                
                fig_infrastructure.update_layout(
                    title='Payment Infrastructure Growth',
                    xaxis_title='Period',
                    yaxis_title='Number of Machines',
                    template='plotly_white',
                    height=500
                )
                visualizations['payment_infrastructure'] = fig_infrastructure
        
        return visualizations
    
    def generate_financial_insights(self):
        """Generate key insights from financial analysis"""
        
        insights = []
        
        # Interest rate insights
        rate_analysis = self.analyze_interest_rate_dynamics()
        if 'cbr_stats' in rate_analysis:
            cbr_stats = rate_analysis['cbr_stats']
            if cbr_stats['trend'] > 0:
                insights.append(f"Central Bank Rate is on an upward trend (current: {cbr_stats['current_rate']:.2f}%)")
            else:
                insights.append(f"Central Bank Rate is on a downward trend (current: {cbr_stats['current_rate']:.2f}%)")
            
            insights.append(f"CBR volatility: {cbr_stats['volatility']:.2f}% - {'High' if cbr_stats['volatility'] > 2 else 'Moderate' if cbr_stats['volatility'] > 1 else 'Low'} volatility")
        
        if 'commercial_rates' in rate_analysis:
            comm_rates = rate_analysis['commercial_rates']
            insights.append(f"Current interest rate spread: {comm_rates['current_spread']:.2f}%")
            
            if comm_rates['current_spread'] > comm_rates['average_spread']:
                insights.append("Interest rate spread is above historical average - banking sector profitability may be improving")
            else:
                insights.append("Interest rate spread is below historical average - increased competition in banking sector")
        
        # Money market insights
        operations = self.analyze_money_market_operations()
        if 'repo_operations' in operations:
            repo_ops = operations['repo_operations']
            if repo_ops['trend'] > 0:
                insights.append("Repo market activity is increasing - indicating active liquidity management")
            else:
                insights.append("Repo market activity is declining - may indicate reduced liquidity pressures")
        
        # Payment systems insights
        payments = self.analyze_payment_systems()
        if 'atm_infrastructure' in payments:
            atm_growth = payments['atm_infrastructure']['growth_rate']
            if atm_growth > 0:
                insights.append(f"ATM infrastructure is expanding (growth rate: {atm_growth:.1f} units/period)")
            else:
                insights.append("ATM infrastructure growth has slowed or is contracting")
        
        if 'pos_infrastructure' in payments:
            pos_growth = payments['pos_infrastructure']['growth_rate']
            if pos_growth > 0:
                insights.append(f"POS infrastructure is growing rapidly (growth rate: {pos_growth:.1f} units/period)")
        
        return insights

# Initialize analyzer
analyzer = BankingFinancialAnalyzer()

print("🏦 Banking and Financial Markets Analysis Notebook")
print("=" * 55)
print("Comprehensive analysis of Kenya's banking sector and financial markets")
print("including interest rates, money market operations, and payment systems.")
print()

# Load and analyze data
print("📊 Loading financial data...")
analyzer.load_financial_data()
analyzer.clean_financial_data()

print("\n🔍 Analyzing interest rate dynamics...")
rate_analysis = analyzer.analyze_interest_rate_dynamics()

print("\n💰 Analyzing money market operations...")
operations_analysis = analyzer.analyze_money_market_operations()

print("\n💳 Analyzing payment systems...")
payments_analysis = analyzer.analyze_payment_systems()

print("\n🔍 Generating financial insights...")
insights = analyzer.generate_financial_insights()
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\n📈 Creating visualizations...")
visualizations = analyzer.create_financial_visualizations()
print(f"Generated {len(visualizations)} visualizations")

print("\n✅ Banking and Financial Markets Analysis Complete!")
print("Key outputs:")
print("- Interest rate trend analysis")
print("- Money market operations assessment")
print("- Payment systems infrastructure tracking")
print("- Financial stability indicators")
print("- Policy effectiveness metrics")
print("- Interactive financial visualizations")